In [2]:
import pandas as pd

!pip install stocksymbol -q

from google.colab import drive
drive.mount('drive')


!pip install gnews -q


## ML
! pip install transformers

!pip install datasets -q
!pip install optuna -q

Mounted at drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.8/115.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.1/517.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.15.0 requires requests>=2.27.1, but you have requests 2.26.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.26.0 which is incompatible.
tweepy 4.14.0 requires requests<3,>=2.27.

## Parsing all cycle

### Classical parser


In [4]:
!pip install dateparser -q

## Source https://github.com/kotartemiy/pygooglenews/blob/master/pygooglenews/__init__.py
import feedparser
from bs4 import BeautifulSoup
import urllib
from dateparser import parse as parse_date
import requests



class GoogleNews:
    def __init__(self, lang = 'en', country = 'USA'):
        self.lang = lang.lower()
        self.country = country.upper()
        self.BASE_URL = 'https://news.google.com/rss'

    def __top_news_parser(self, text):
        """Return subarticles from the main and topic feeds"""
        try:
            bs4_html = BeautifulSoup(text, "html.parser")
            # find all li tags
            lis = bs4_html.find_all('li')
            sub_articles = []
            for li in lis:
                try:
                    sub_articles.append({"url": li.a['href'],
                                         "title": li.a.text,
                                         "publisher": li.font.text})
                except:
                    pass
            return sub_articles
        except:
            return text

    def __ceid(self):
        """Compile correct country-lang parameters for Google News RSS URL"""
        return '?ceid={}:{}&hl={}&gl={}'.format(self.country,self.lang,self.lang,self.country)

    def __add_sub_articles(self, entries):
        for i, val in enumerate(entries):
            if 'summary' in entries[i].keys():
                entries[i]['sub_articles'] = self.__top_news_parser(entries[i]['summary'])
            else:
                entries[i]['sub_articles'] = None
        return entries

    def __scaping_bee_request(self, api_key, url):
        response = requests.get(
            url="https://app.scrapingbee.com/api/v1/",
            params={
                "api_key": api_key,
                "url": url,
                "render_js": "false"
            }
        )
        if response.status_code == 200:
            return response
        if response.status_code != 200:
            raise Exception("ScrapingBee status_code: "  + str(response.status_code) + " " + response.text)

    def __parse_feed(self, feed_url, proxies=None, scraping_bee = None):

        if scraping_bee and proxies:
            raise Exception("Pick either ScrapingBee or proxies. Not both!")

        if proxies:
            r = requests.get(feed_url, proxies = proxies)
        else:
            r = requests.get(feed_url)

        if scraping_bee:
            r = self.__scaping_bee_request(url = feed_url, api_key = scraping_bee)
        else:
            r = requests.get(feed_url)


        if 'https://news.google.com/rss/unsupported' in r.url:
            raise Exception('This feed is not available')

        d = feedparser.parse(r.text)

        if not scraping_bee and not proxies and len(d['entries']) == 0:
            d = feedparser.parse(feed_url)

        return dict((k, d[k]) for k in ('feed', 'entries'))

    def __search_helper(self, query):
        return urllib.parse.quote_plus(query)

    def __from_to_helper(self, validate=None):
        try:
            validate = parse_date(validate).strftime('%Y-%m-%d')
            return str(validate)
        except:
            raise Exception('Could not parse your date')



    def top_news(self, proxies=None, scraping_bee = None):
        """Return a list of all articles from the main page of Google News
        given a country and a language"""
        d = self.__parse_feed(self.BASE_URL + self.__ceid(), proxies=proxies, scraping_bee=scraping_bee)
        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

    def topic_headlines(self, topic: str, proxies=None, scraping_bee=None):
        """Return a list of all articles from the topic page of Google News
        given a country and a language"""
        #topic = topic.upper()
        if topic.upper() in ['WORLD', 'NATION', 'BUSINESS', 'TECHNOLOGY', 'ENTERTAINMENT', 'SCIENCE', 'SPORTS', 'HEALTH']:
            d = self.__parse_feed(self.BASE_URL + '/headlines/section/topic/{}'.format(topic.upper()) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        else:
            d = self.__parse_feed(self.BASE_URL + '/topics/{}'.format(topic) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        if len(d['entries']) > 0:
            return d
        else:
            raise Exception('unsupported topic')

    def geo_headlines(self, geo: str, proxies=None, scraping_bee=None):
        """Return a list of all articles about a specific geolocation
        given a country and a language"""
        d = self.__parse_feed(self.BASE_URL + '/headlines/section/geo/{}'.format(geo) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

    def search(self, query: str, helper = True, when = None, from_ = None, to_ = None, proxies=None, scraping_bee=None):
        """
        Return a list of all articles given a full-text search parameter,
        a country and a language
        :param bool helper: When True helps with URL quoting
        :param str when: Sets a time range for the artiles that can be found
        """

        if when:
            query += ' when:' + when

        if from_ and not when:
            from_ = self.__from_to_helper(validate=from_)
            query += ' after:' + from_

        if to_ and not when:
            to_ = self.__from_to_helper(validate=to_)
            query += ' before:' + to_

        if helper == True:
            query = self.__search_helper(query)

        search_ceid = self.__ceid()
        search_ceid = search_ceid.replace('?', '&')

        d = self.__parse_feed(self.BASE_URL + '/search?q={}'.format(query) + search_ceid, proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

In [5]:
import datetime
gn = GoogleNews(lang = 'en')

## function por parsing several days
##Created to break limit of 100 publications per request by Googgle
##Credits to https://stackoverflow.com/questions/72571345/googlenews-date-range
def get_news(search):
   stories = []
   start_date = datetime.date(2020,1,1)
   end_date = datetime.date(2023,9,1)
   print(start_date, end_date)
   delta = datetime.timedelta(days=5)  ## timestamp object to datetime
   date_list = pd.date_range(start_date, end_date).tolist()
   ##print(date_list)

   for date in date_list[:-1]:
    x = (date).strftime('%Y-%m-%d')
    x1 = (date+delta).strftime('%Y-%m-%d')
    ##print(x1)
    result = gn.search(search, from_=(date).strftime('%Y-%m-%d'), to_=(date+delta).strftime('%Y-%m-%d'))
    newsitem = result['entries']

    for item in newsitem:
        story = {
            'title':item.title,
            'link':item.link,
            'published':item.published
        }
        stories.append(story)
   print("Finished parsing of:", search,"for dates:", start_date, '-', end_date )
   return stories

### Parsing

In [ ]:
from stocksymbol import StockSymbol

## Getting stock tickers from S&P 500 index
api_key = '4231f700-99e1-42ec-b8a8-03b27d3695f3'
ss = StockSymbol(api_key)
symbol_list_us = ss.get_symbol_list(index="SPX")
Tickers = pd.DataFrame(symbol_list_us)

Tickers.to_csv('symbol_list_selected.csv')
!cp symbol_list_selected.csv "/content/drive/MyDrive/Colab Notebooks/Data_paper"

In [ ]:
## Read list of stock tickers

symbol_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_paper/symbol_list_selected.csv')

In [ ]:
symbol_list

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,symbol,shortName,longName,exchange,market,quoteType,Parsed
0,0,0,0,0,0,AAPL,apple,Apple Inc.,NASDAQ,us_market,EQUITY,1
1,1,1,1,1,1,MSFT,microsoft,Microsoft Corporation,NASDAQ,us_market,EQUITY,1
2,2,2,2,2,2,GOOGL,alphabet,Alphabet Inc.,NASDAQ,us_market,EQUITY,1
3,3,3,3,3,3,GOOG,alphabet,Alphabet Inc.,NASDAQ,us_market,EQUITY,1
4,4,4,4,4,4,AMZN,amazon,"Amazon.com, Inc.",NASDAQ,us_market,EQUITY,1
...,...,...,...,...,...,...,...,...,...,...,...,...
499,499,499,499,499,499,PENN,penn-national-gaming,"Penn National Gaming, Inc.",NASDAQ,us_market,EQUITY,0
500,500,500,500,500,500,NLSN,nielsen-hldgs-plc,Nielsen N.V.,NYSE,us_market,EQUITY,0
501,501,501,501,501,501,PVH,pvh,PVH Corp.,NYSE,us_market,EQUITY,0
502,502,502,502,502,502,ALK,alaska-air,"Alaska Air Group, Inc.",NYSE,us_market,EQUITY,0


In [ ]:
##Given that parsing was done in steps we manually adjust already parsed stock tickers
symbol_list.loc[:40, 'Parsed'] =1
symbol_list.to_csv('symbol_list_selected.csv')
!cp symbol_list_selected.csv "/content/drive/MyDrive/Colab Notebooks/Data_paper"

In [ ]:
## init dataset to write to

News_dataset = pd.DataFrame(list())
News_dataset.to_csv('News_dataset.csv')
##!cp News_dataset.csv "/content/drive/MyDrive/Colab Notebooks/Data_paper"

In [ ]:
News_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_paper/News_dataset.csv')

In [ ]:
News_dataset

,,,,,Unnamed: 0
0,Apple and Imagination Technologies Bury the Hatchet; Upcoming iPhones May Feature New PowerVR GPUs - Wccftech,https://news.google.com/rss/articles/CBMiQ2h0dHBzOi8vd2NjZnRlY2guY29tL2FwcGxlLWltYWdpbmF0aW9uLXRlY2hub2xvZ2llcy1uZXctYmVnaW5uaW5ncy_SAUdodHRwczovL3djY2Z0ZWNoLmNvbS9hcHBsZS1pbWFnaW5hdGlvbi10ZWNobm9sb2dpZXMtbmV3LWJlZ2lubmluZ3MvYW1wLw?oc=5,"Thu, 02 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
1,"Apple's stock could be worth $100 more in 2020, tech analyst Gene Munster predicts - CNBC",https://news.google.com/rss/articles/CBMiZmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjAvMDEvMDEvYXBwbGUtc3RvY2stY291bGQtYmUtd29ydGgtMTAwLW1vcmUtaW4tMjAyMC1nZW5lLW11bnN0ZXItcHJlZGljdHMuaHRtbNIBamh0dHBzOi8vd3d3LmNuYmMuY29tL2FtcC8yMDIwLzAxLzAxL2FwcGxlLXN0b2NrLWNvdWxkLWJlLXdvcnRoLTEwMC1tb3JlLWluLTIwMjAtZ2VuZS1tdW5zdGVyLXByZWRpY3RzLmh0bWw?oc=5,"Wed, 01 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
2,Apple restores Imagination GPU chip agreement after public dispute ... - 9to5Mac,https://news.google.com/rss/articles/CBMiO2h0dHBzOi8vOXRvNW1hYy5jb20vMjAyMC8wMS8wMS9hcHBsZS1pbWFnaW5hdGlvbi1hZ3JlZW1lbnQv0gEA?oc=5,"Wed, 01 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
3,"Demolition underway at Apple Valley, Summitview; District looks at grade configurations - Yakima Herald-Republic",https://news.google.com/rss/articles/CBMirwFodHRwczovL3d3dy55YWtpbWFoZXJhbGQuY29tL25ld3MvbG9jYWwvZGVtb2xpdGlvbi11bmRlcndheS1hdC1hcHBsZS12YWxsZXktc3VtbWl0dmlldy1kaXN0cmljdC1sb29rcy1hdC1ncmFkZS1jb25maWd1cmF0aW9ucy9hcnRpY2xlXzEyMzdkYTFjLTBjNDItNTc0YS1hYTgxLTNlOTk1OGNjZDM1Ni5odG1s0gGzAWh0dHBzOi8vd3d3Lnlha2ltYWhlcmFsZC5jb20vbmV3cy9sb2NhbC9kZW1vbGl0aW9uLXVuZGVyd2F5LWF0LWFwcGxlLXZhbGxleS1zdW1taXR2aWV3LWRpc3RyaWN0LWxvb2tzLWF0LWdyYWRlLWNvbmZpZ3VyYXRpb25zL2FydGljbGVfMTIzN2RhMWMtMGM0Mi01NzRhLWFhODEtM2U5OTU4Y2NkMzU2LmFtcC5odG1s?oc=5,"Wed, 01 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
4,Want Fresh Fruit in Winter? Try Crab Apple Trees - Lake Minnetonka Magazine,https://news.google.com/rss/articles/CBMiS2h0dHBzOi8vbGFrZW1pbm5ldG9ua2FtYWcuY29tL3dhbnQtZnJlc2gtZnJ1aXQtd2ludGVyLXRyeS1jcmFiLWFwcGxlLXRyZWVzL9IBAA?oc=5,"Thu, 29 Jun 2023 16:30:05 GMT",NASDAQ,AAPL
...,...,...,...,...,...
118916,Bret Stephens: How Do We Manage China's Decline? GUEST ... - Baltimore Sun,https://news.google.com/rss/articles/CBMicGh0dHBzOi8vd3d3LmJhbHRpbW9yZXN1bi5jb20vb3Bpbmlvbi9vcC1lZC9icy1lZC1vcC1jaGluYS1kZWNsaW5lLTIwMjMwOTAxLXl0a2J1N21hd3JndnJrcDJxcW42cmN3dHZtLXN0b3J5Lmh0bWzSAQA?oc=5,"Fri, 01 Sep 2023 07:00:00 GMT",NASDAQ,QCOM
118917,Unveiling The Connected Britain Awards 2023 Shortlist - VoIp.Review,https://news.google.com/rss/articles/CBMiUWh0dHBzOi8vdm9pcC5yZXZpZXcvMjAyMy8wOS8wNC91bnZlaWxpbmctY29ubmVjdGVkLWJyaXRhaW4tYXdhcmRzLTIwMjMtc2hvcnRsaXN0L9IBAA?oc=5,"Mon, 04 Sep 2023 07:00:00 GMT",NASDAQ,QCOM
118918,Closing the Book: Microsoft WordPad 3 Decade Story is Ending - Techweez,https://news.google.com/rss/articles/CBMiP2h0dHBzOi8vdGVjaHdlZXouY29tLzIwMjMvMDkvMDIvbWljcm9zb2Z0LWtpbGxpbmctd29yZHBhZC1zb29uL9IBAA?oc=5,"Sat, 02 Sep 2023 07:00:00 GMT",NASDAQ,QCOM
118919,Monthly Merger Review Snapshot - Law360,https://news.google.com/rss/articles/CBMiRmh0dHBzOi8vd3d3LmxhdzM2MC5jb20vYXJ0aWNsZXMvMTcxNzIwNi9tb250aGx5LW1lcmdlci1yZXZpZXctc25hcHNob3TSAStodHRwczovL3d3dy5sYXczNjAuY29tL2FtcC9hcnRpY2xlcy8xNzE3MjA2?oc=5,"Fri, 01 Sep 2023 07:00:00 GMT",NASDAQ,QCOM


In [ ]:
del News_dataset

In [ ]:
Symbol_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_paper/symbol_list_selected.csv')

##We parse only ~50 top tickers
Symbol_list =Symbol_list.head(51)

##parsing function to collect news from Google News
for index, row in Symbol_list.iterrows():
  if row['Parsed'] == 0:
      ## Run function to get publications from the short name of company
      List_of_news = get_news(row['shortName'])

      Data_frac = pd.DataFrame(List_of_news)

      ## Append additional columns
      Data_frac['exchange'] = row['exchange']
      Data_frac['symbol'] = row['symbol']
      Data_frac.to_csv('/content/drive/MyDrive/Colab Notebooks/Data_paper/News_dataset.csv', mode='a', header=False, escapechar='\\')


      ## Run function to get publications from the ticker of company
      List_of_news2 = get_news(row['symbol'])
      Data_frac = pd.DataFrame(List_of_news2)

      ## Append additional columns
      Data_frac['exchange'] = row['exchange']
      Data_frac['symbol'] = row['symbol']
      Data_frac.to_csv('/content/drive/MyDrive/Colab Notebooks/Data_paper/News_dataset.csv', mode='a', header=False, escapechar='\\')


  else:
    print(row['shortName'], 'is already parced')

apple is already parced
microsoft is already parced
alphabet is already parced
alphabet is already parced
amazon is already parced
tesla is already parced
meta-platforms is already parced
berkshire-hathaway is already parced
nvidia is already parced
jpmorgan-chase is already parced
visa is already parced
unitedhealth is already parced
johnson-and-johnson is already parced
walmart is already parced
home-depot is already parced
bank-of-america is already parced
procter-and-gamble is already parced
mastercard is already parced
pfizer is already parced
exxon is already parced
walt-disney is already parced
coca-cola is already parced
cisco is already parced
chevron is already parced
broadcom is already parced
adobe is already parced
pepsico is already parced
thermo-fisher-scientific is already parced
eli-lilly is already parced
abbvie is already parced
accenture is already parced
comcast is already parced
nike is already parced
oracle is already parced
netflix is already parced
wells-fargo 

### Data Overview

In [ ]:
News_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_paper/News_dataset.csv', sep=",")

In [ ]:
News_dataset.row.str.extract('(?P<Title>.*?) - ((?P<Source>[A-Z ]*$)|(?P<county>.*?), (?P<state_code>[A-Z]{2}$))')

In [ ]:
test1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_paper/News_datasetHeader.csv', header=0, sep=",")


In [ ]:
import plotly.express as px
to_plot = test1.groupby(test1[' Tiker'])

figa = px.bar(to_plot['Index'].count(), title='Number of publications from 01.2020 to 09.2023')
figa.update_layout(showlegend=False)
figa.update_xaxes(title='Ticker', visible=True, showticklabels=True)

In [ ]:
import yfinance as yf



/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.17) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
test1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_paper/News_datasetHeader.csv', header=0, sep=",")

In [ ]:
test1

,Index,Title,Link,Date; Exchange,Tiker
0,Apple and Imagination Technologies Bury the Ha...,https://news.google.com/rss/articles/CBMiQ2h0d...,"Thu, 02 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
1,Apple's stock could be worth $100 more in 2020...,https://news.google.com/rss/articles/CBMiZmh0d...,"Wed, 01 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
2,Apple restores Imagination GPU chip agreement ...,https://news.google.com/rss/articles/CBMiO2h0d...,"Wed, 01 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
3,"Demolition underway at Apple Valley, Summitvie...",https://news.google.com/rss/articles/CBMirwFod...,"Wed, 01 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
4,Want Fresh Fruit in Winter? Try Crab Apple Tre...,https://news.google.com/rss/articles/CBMiS2h0d...,"Thu, 29 Jun 2023 16:30:05 GMT",NASDAQ,AAPL
...,...,...,...,...,...
85919,"After a Texas school shooting, conservatives b...",https://news.google.com/rss/articles/CBMiamh0d...,"Mon, 18 Jul 2022 07:00:00 GMT",NYSE,ABT
85920,Aussies make major announcement on Pat Cummins...,https://news.google.com/rss/articles/CBMiYmh0d...,"Sun, 17 Jul 2022 07:00:00 GMT",NYSE,ABT
85921,"ERCOT Asked and Texas Listened, and the Power ...",https://news.google.com/rss/articles/CBMiZ2h0d...,"Fri, 15 Jul 2022 07:00:00 GMT",NYSE,ABT
85922,Television: ‘Yellowstone’ should have received...,https://news.google.com/rss/articles/CBMicGh0d...,"Sun, 17 Jul 2022 07:00:00 GMT",NYSE,ABT


In [ ]:
##List of companies in dataset

Companies = test1[' Tiker'].unique()
Companies = Companies.tolist()

In [ ]:
##Get financial data
from stocksymbol import StockSymbol
Stock_data = pd.DataFrame()

for comp in Companies:
  ticker = yf.Ticker(comp)
  hist = ticker.history(period="4y")
  hist['symbol'] = comp
  Stock_data = Stock_data.append(hist)

In [ ]:
Stock_data.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'symbol', 'Adj Close'],
      dtype='object')

In [ ]:
Stock_data.to_csv('Stock_data.csv')
!cp Stock_data.csv "/content/drive/MyDrive/Colab Notebooks/Data_paper"

In [ ]:
Stock_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,symbol,Adj Close
Date,,,,,,,,,
2020-09-30 00:00:00-04:00,111.769705,115.178098,111.602725,113.753838,142675200.0,0.0,0.0,AAPL,NaN
2020-10-01 00:00:00-04:00,115.551368,115.629950,113.773506,114.716461,116120400.0,0.0,0.0,AAPL,NaN
2020-10-02 00:00:00-04:00,110.885700,113.321673,110.227598,111.013390,144712000.0,0.0,0.0,AAPL,NaN
2020-10-05 00:00:00-04:00,111.887590,114.578941,111.533981,114.431602,106243800.0,0.0,0.0,AAPL,NaN
2020-10-06 00:00:00-04:00,113.645814,114.058363,110.257070,111.150917,161498200.0,0.0,0.0,AAPL,NaN
...,...,...,...,...,...,...,...,...,...
2023-09-25 00:00:00-04:00,97.970001,98.209999,97.160004,97.470001,4096300.0,0.0,0.0,ABT,NaN
2023-09-26 00:00:00-04:00,97.050003,97.529999,96.029999,96.230003,3967700.0,0.0,0.0,ABT,NaN
2023-09-27 00:00:00-04:00,96.589996,96.940002,94.760002,95.529999,5391000.0,0.0,0.0,ABT,NaN


In [ ]:
import plotly.express as px
fig = px.line(Stock_data,color='symbol', y='Close')
fig.update_yaxes(title='Adj. Close', visible=True, showticklabels=True)

##Project

In [ ]:
##import news DataFrame
names = ["index","Title", 'link', 'Date', 'Ex.', 'Company']
News_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_paper/News_dataset.csv', header=0, names = names, sep=",")
News_dataset

,index,Title,link,Date,Ex.,Company
0,0,Apple and Imagination Technologies Bury the Ha...,https://news.google.com/rss/articles/CBMiQ2h0d...,"Thu, 02 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
1,1,Apple's stock could be worth $100 more in 2020...,https://news.google.com/rss/articles/CBMiZmh0d...,"Wed, 01 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
2,2,Apple restores Imagination GPU chip agreement ...,https://news.google.com/rss/articles/CBMiO2h0d...,"Wed, 01 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
3,3,"Demolition underway at Apple Valley, Summitvie...",https://news.google.com/rss/articles/CBMirwFod...,"Wed, 01 Jan 2020 08:00:00 GMT",NASDAQ,AAPL
4,4,Want Fresh Fruit in Winter? Try Crab Apple Tre...,https://news.google.com/rss/articles/CBMiS2h0d...,"Thu, 29 Jun 2023 16:30:05 GMT",NASDAQ,AAPL
...,...,...,...,...,...,...
6899116,45517,【卓球男子】2024年パリ五輪 日本代表選考ポイント（2023年8月31日現在）｜テレビ東京...,https://news.google.com/rss/articles/CBMiP2h0d...,"Thu, 31 Aug 2023 07:00:00 GMT",NASDAQ,TXN
6899117,45518,人気ベーカリー｜2023年9月 2日｜出没！アド街ック天国：テレビ東京 - テレビ東京・ＢＳ...,https://news.google.com/rss/articles/CBMiQ2h0d...,"Sat, 02 Sep 2023 07:00:00 GMT",NASDAQ,TXN
6899118,45519,五線紙｜2023年9月 2日｜出没！アド街ック天国：テレビ東京 - テレビ東京・ＢＳテレ東 ...,https://news.google.com/rss/articles/CBMiQ2h0d...,"Sat, 02 Sep 2023 07:00:00 GMT",NASDAQ,TXN
6899119,45520,“敬護”の精神で全面サポート～頼れるスーパー介護施設！：読んで ... - テレ東プラス,https://news.google.com/rss/articles/CBMiQGh0d...,"Thu, 31 Aug 2023 07:00:00 GMT",NASDAQ,TXN


In [ ]:
!pip install datasets -q
!pip install optuna -q

In [ ]:
import numpy as np
import spacy
from sklearn.cluster import DBSCAN
import yfinance as yf
from tqdm import tqdm
tqdm.pandas()

from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import BDay

import yfinance as yf

import tensorflow as tf
import torch



from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


import optuna
from optuna.samplers import TPESampler

random_state=42

class ML_build:

  ## init class
  def __init__(self):
    return

  ## assign news dataframe into class
  def news_df(self, df):


    self.df_news = df.copy()
    self.df_news.loc[:,'Date'] = pd.to_datetime(self.df_news.loc[:,'Date'])
    return

  ## data cleaning from duplicates, other languages
  def data_preparation(self, title_column = 'Title',
                       drop_dup=True, sources_sep = True, Non_eng = 'drop'):
    ## Drop duplicates. Requiered step Given that we parsed for each company twice
    if drop_dup ==True:
      self.df_news = self.df_news.drop_duplicates()
    else:
      print("duplicates were not excluded")

    ## Drop Non-english news
    if Non_eng == 'drop':
      def isEnglish(s):
        return s.isascii()

      self.df_news = self.df_news[self.df_news[title_column].apply(isEnglish) == True]
    else:
      print("other languages were not excluded")


    if 'Source' not in self.df_news:
      self.df_news[['Title', 'Source']] = self.df_news['Title'].str.rsplit(' - ', n=1, expand=True)
    else:
      pass
    return


  ## finding important new by clusterisation based on rare words
  def first_story_flag(self, Light_model = True):
    if Light_model == False:
      ##advanced vectorisation. Does not support large df.
      sent_vecs = {}
      docs = []
      spacy.cli.download('en_core_web_lg')
      nlp = spacy.load('en_core_web_lg')
      for title in tqdm(self.df_news.Title.head(1000)):
        doc = nlp(title)
        docs.append(doc)
        sent_vecs.update({title: doc.vector})
        sentences = list(sent_vecs.keys())
        vectors = list(sent_vecs.values())
      x = np.array(vectors)
      ## search for optimal eps
      n_classes = {}
      for i in tqdm(np.arange(0.001, 1, 0.002)):
        dbscan = DBSCAN(eps=i, min_samples=2, metric='cosine').fit(x)
        n_classes.update({i: len(pd.Series (dbscan. labels_) .value_counts ())})
      print(n_classes)
      res_list = []
      for item in dbscan.labels_:
          if item not in res_list:
              res_list.append(item)
      print(res_list)


      ## Get most sensetive classification
      optimal_eps   = max(n_classes, key=n_classes.get)
      dbscan = DBSCAN(eps=optimal_eps, min_samples=2, metric='cosine').fit(x)
      print(dbscan.labels_, self.df_news.Title)

      result_dbscan = pd.DataFrame({'DBSCAN': dbscan.labels_,
                                    'sent': self.df_news.Title})
      self.df_news = self.df_news.merge(result_dbscan,
                                        left_on='Title', right_on='sent')
      ##self.df_news = self.df_news.rename(columns={"A": "a", "B": "c"})
      return


    else:
      ## run TF-ID model Based on http://ai.intelligentonlinetools.com/ml/text-clustering-word-embedding-machine-learning/
      from sklearn.feature_extraction.text import TfidfVectorizer

      vec = TfidfVectorizer( stop_words='english')##,
                      ##ngram_range=(1, 2))
      x = vec.fit_transform(self.df_news.head(1000).Title)
      ## search for optimal eps
      n_classes = {}
      for i in tqdm(np.arange(0.001, 1, 0.002)):
        dbscan = DBSCAN(eps=i, min_samples=2, metric='cosine').fit(x)
        n_classes.update({i: len(pd.Series (dbscan. labels_) .value_counts ())})
      print(n_classes)

      ## init full dataset

      Final_DBSCAN_df = pd.DataFrame()
      for company in tqdm(self.df_news['Company'].unique()):
        df_train = self.df_news[self.df_news['Company'] == company]
        vec = TfidfVectorizer( stop_words='english')##,
                        ##ngram_range=(1, 2))
        x = vec.fit_transform(self.df_news.Title)

        ## Get most sensetive classification
        optimal_eps   = max(n_classes, key=n_classes.get)
        dbscan = DBSCAN(eps=optimal_eps, min_samples=2, metric='cosine').fit(x)
        ##print(dbscan.labels_, self.df_news.Title)

        result_dbscan = pd.DataFrame({'DBSCAN': dbscan.labels_,
                                      'sent': self.df_news.Title})

        Final_DBSCAN_df = pd.concat([Final_DBSCAN_df, result_dbscan])
      ##print(Final_DBSCAN_df)
      self.df_news = self.df_news.merge(Final_DBSCAN_df,
                                        left_on='Title', right_on='sent')
      ##self.df_news = self.df_news.rename(columns={"A": "a", "B": "c"})


      ## Get flag for first row in class
      Story_list = []
      def first_occurance(value):
        if value in Story_list:
          return 0
        else:
          Story_list.append(value)
          return 1

      self.df_news['First_story'] = self.df_news['DBSCAN'].apply(first_occurance)


      return


  ## inital semtiment analysis


    return

  def BERT_sentiment_test(self, column = 'Title' ):
    from datasets import Dataset
    from transformers.pipelines.pt_utils import KeyDataset
    from transformers import pipeline

    nlp = pipeline(task='sentiment-analysis',
               model='ProsusAI/finbert', device=-1, batch_size=100)


    result = []

    dataset = Dataset.from_pandas(self.df_news)

    for out in tqdm(nlp(KeyDataset(dataset, 'Title'))):
      result.append(out)
    print(out)
    result = pd.DataFrame(result)
    result.cou

    self.df_news= self.df_news.join(result.set_axis(self.df_news.index))

    ##self.df_news['FinBERT'] = result.label
    ##self.df_news['Fin_BERT_score'] = result.score
    return

  def BERT_quality_test(self, column = 'Title' ):
    from datasets import Dataset
    from transformers.pipelines.pt_utils import KeyDataset
    from transformers import pipeline

    nlp = pipeline("text-classification", model="ikoghoemmanuell/finetuned_fake_news_roberta",
                   device=-1, batch_size=100)


    result = []

    dataset = Dataset.from_pandas(self.df_news)

    for out in tqdm(nlp(KeyDataset(dataset, 'Title'))):
      result.append(out)
    result = pd.DataFrame(result)

    self.df_news= self.df_news.join(result.set_axis(self.df_news.index))


    ##self.df_news['FakeBERT'] = result.label
    ##self.df_news['FakeBERT_score'] = result.score
    return

  def BERT_NER_test(self, column = 'Title' ):
    from datasets import Dataset
    from transformers.pipelines.pt_utils import KeyDataset
    from transformers import pipeline

    nlp = pipeline(task='ner',
               model='dslim/bert-base-NER', device=0, batch_size=100)

    result = []
    dataset = Dataset.from_pandas(self.df_news)

    for out in tqdm(nlp(KeyDataset(dataset, 'Title'))):
      out1 = [i['word'] for i in out]
      out2 = [i['entity'] for i in out]

      output = {}
      ##for d in out:
      ##  output.update(d)
      ##result.append(out)
      output =dict.fromkeys(out2, out1)
      result.append(output)

    result = pd.DataFrame(result)

    ##self.df_news = pd.concat([self.df_news, result], axis=1, ignore_index=True)

    self.df_news= self.df_news.join(result.set_axis(self.df_news.index))

    return





  def save_BERT(self):
    self.df_news.to_csv('BERT_project_results.csv')
    !cp BERT_project_results.csv "/content/drive/MyDrive/Colab Notebooks/Data_paper"
    return

  def load_BERT(self):
    del self.df_news

    ##Currently we work with 100000 publications (less than 2% of dataset)
    ## It is done to ensure swift working of the functions bellow.
    self.df_news = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_paper/BERT_project_results.csv',
                               low_memory=False, nrows=100000)




    ##self.df_news = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data_paper/BERT_project_results.csv',
    ##                        low_memory=False)


    return





  ##Get Source feature, adjust time of publication
  def time_publication_adjustment(self, naive):
    ## assume lag of google news is several hours
    self.df_news['Date'] = pd.to_datetime(self.df_news['Date'])

    if naive ==True:
      ## move all publications of Saturday and Sunday to Monday
      print(self.df_news['Date']) ##+ pd.offsets.CustomBusinessDay(n=1, calendar=USFederalHolidayCalendar()))
      ##self.df_news.loc[:,'Date'] = self.df_news['Date']+ pd.offsets.BusinessDay(0)

      def offset_date(start, offset):
        return start + pd.offsets.CustomBusinessDay(n=offset, calendar=USFederalHolidayCalendar())

      offset = 0
      self.df_news['Date']= self.df_news['Date'].apply(lambda x: offset_date(x,
                                           offset))
      self.df_news['Date'] = self.df_news['Date'].dt.date
      print(self.df_news['Date'])

      self.df_news['Source'] = self.df_news['Title'].str.split(' - ').str[1]

      return
    else:
      from sklearn.model_selection import GridSearchCV

      self.df_news['Date'] = pd.to_datetime(self.df_news['Date'])

      ##self.df_news['Source'] = self.df_news['Title'].str.split(' - ').str[1]


      self.Adjustment = {}


      for source in self.df_news['Source']:

          df_train = self.df_news[self.df_news['Source'] == source]






          def objective(trial):

            lag = trial.suggest_categorical('lag' , [0, 0.25, 0.5, 0.75])
            print(round(lag*4))
            def offset_date(df, offset = 0, offset2 = lag*4):
              try:
                first = df['Date']
                second = pd.offsets.CustomBusinessDay(n=offset, calendar=USFederalHolidayCalendar())
                third = pd.offsets.DateOffset(day=offset2)
                return first + second - third
              except:
                return df['Date'] + pd.offsets.CustomBusinessDay(n=offset, calendar=USFederalHolidayCalendar())

            offset = 0

            df_train['Date']= df_train.apply(lambda x: offset_date(x), axis=1)
            df_train['Date'] = df_train['Date'].dt.date


            def get_current_prediction(df_sources):
              New_class = ML_build()
              New_class.news_df(df_sources)
              New_class.One_hot_encode()
              ##print(New_class.df_news)

              New_class.import_stock_data()
              New_class.prepare_final()
              New_class.train_test_split()
              ##print(New_class.X_train, New_class.y_train)
              model = RandomForestClassifier().fit(New_class.X_train, New_class.y_train)
              y_pred = model.predict(New_class.X_train)
              ##print(New_class.y_train, y_pred)
              return f1_score(New_class.y_train, y_pred)

            result = get_current_prediction(df_train)
            ##print(df_train)
            print(source, lag, result)

            return result

          sampler = TPESampler(seed=42)
          study = optuna.create_study(direction='maximize', sampler=sampler)
          study.optimize(objective, n_trials=5)

          self.Adjustment[source] = study.best_trial.params['lag']
          print(self.Adjustment)



      return

  ## request stock market data from
  def import_stock_data(self):
    Companies = self.df_news['Company'].unique()
    Companies = Companies.tolist()

    Stock_data = pd.DataFrame()

    ##Get index returns
    ticker = yf.Ticker('^GSPC')
    SP500 = ticker.history(period="4y")
    SP500["Returns"] = (SP500.Close - SP500.Open)/SP500.Open


    ##stock data for each company
    for comp in Companies:
      ticker = yf.Ticker(comp)
      hist = ticker.history(period="4y")
      ## append symbol
      hist['symbol'] = comp

      ##Add index returns for each company
      hist['SP500_returns'] = SP500["Returns"]
      hist['SP500_returns_yesterday'] = SP500["Returns"].shift(-1)

      Stock_data = Stock_data.append(hist)

    ## Add SP500
    ##ticker = yf.Ticker('^GSPC')
    ##SP500 = ticker.history(period="4y")
    ##Stock_data = pd.concat([SP500, Stock_data])


    ##Create returns variable
    Stock_data["Returns"] = (Stock_data.Close - Stock_data.Open)/Stock_data.Open
    Stock_data["Returns_binary"] = 0
    Stock_data.loc[Stock_data["Returns"]>0,["Returns_binary"]] = 1
    ## Create previos returns feature
    Stock_data["Returns_yesterday"] = Stock_data["Returns"].shift(-1)

    ##Technical transformations
    Stock_data =Stock_data.reset_index()
    Stock_data.loc[:,'Date'] = pd.to_datetime(Stock_data.loc[:,'Date'])
    Stock_data['Date'] = Stock_data['Date'].dt.date

    self.df_stock = Stock_data
    return

  ##function to get features from BERT models, publishers
  def One_hot_encode(self, Interactions = True, OHE = True):
    ##self.df_news.Title.str.rsplit(" - ", n=1, expand=True)
    ##print(self.df_news.columns)
    self.df_news.reset_index(drop=True, inplace=True)

    ##We have too many Sources. We drop all except top 10.
    ##Credit to https://stackoverflow.com/questions/32511061/remove-low-frequency-values-from-pandas-dataframe

    threshold = 10 # Anything that occurs less than this will be removed.
    value_counts = self.df_news.Source.value_counts() # Entire DataFrame
    to_remove = value_counts[value_counts <= threshold].index
    self.df_news.Source.replace(to_remove, 'Other', inplace=True)

    ## init Encoder
    ohe = OneHotEncoder(categories='auto')

    ##selected columns for encoding
    columns_ohe= ['FinBERT', 'FakeBERT'] ## simple features currently.


    ## NOTE: OneHotEncoder has mamory problems on large dataset
    ## Current roundabout solution


    if OHE == True:
    ##cycle to decrease memory usage.

      feature_arr = ohe.fit_transform(self.df_news[columns_ohe]).toarray()

      feature_labels = ohe.categories_

      feature_labels = np.array(feature_labels, dtype=object).ravel()
      feature_labels = [item for sublist in feature_labels for item in sublist]
      ##print(feature_arr)
      ##print(feature_labels)
      features = pd.DataFrame(feature_arr, columns=feature_labels)

      self.df_news = pd.concat([self.df_news, features], axis=1)

    else:
        cols1 = ['negative', 'neutral', 'positive']
        for col in tqdm(cols1):
          self.df_news[col] = 0
          self.df_news.loc[self.df_news['FinBERT'] == col, col] = 1

        cols2 = ['LABEL_0', 'LABEL_1']
        for col in tqdm(cols2):
          self.df_news[col] = 0
          self.df_news.loc[self.df_news['FakeBERT'] == col, col] = 1
      ##self.df_news[['negative', 'neutral', 'postive']]= pd.get_dummies(
       ##   self.df_news['FinBERT'], dtype=float)
      ##cols2 = ['LABEL_0', 'LABEL_1']
      ##self.df_news[['LABEL_0', 'LABEL_1']]= pd.get_dummies(
       ##   self.df_news['FakeBERT'], dtype=float)

    ##interactions between variables
    if Interactions == True:
      self.df_news['positive*Label_1'] = self.df_news['positive'] * self.df_news['LABEL_1']
      self.df_news['negative*Label_1'] = self.df_news['negative'] * self.df_news['LABEL_1']
      return
    else:
      return

  ## merging dataframes to fit in classifiers
  def prepare_final(self, Interactions = True):

    ## To solve the error when running time_publication_adjustment(naive = False)
    self.df_news['Date'] = pd.to_datetime(self.df_news['Date']).dt.date


    ##Groupby and combine datasets
    New_indicators= self.df_news.groupby(['Company', 'Date']).sum()
    self.df = pd.merge(self.df_stock, New_indicators, left_on=['symbol', 'Date'],
                       right_on=['Company', 'Date'])
    self.df.set_index('Date', inplace = True)

    ## Market uncertanty index
    for date in self.df.index.unique():
      Set = self.df[self.df.index == date]
      Positive_sum = Set['positive'].sum()
      Negative_sum = Set['negative'].sum()
      try:
        Uncert = max(0, 1- Negative_sum / (Positive_sum + Negative_sum))
      except:
        Uncert = 0
      self.df.loc[self.df.index  == date, 'Uncert']  = Uncert

    if Interactions == True:
      self.df['positive*Label_1*Uncert'] = self.df['positive*Label_1'] * self.df.Uncert
      self.df['negative*Label_1*Uncert'] = self.df['negative*Label_1'] * self.df.Uncert
    else:
      pass


    ##drop outside columns: current stock data, leftover columns
    self.df.drop(['Open', 'High', 'Low', 'Close', 'Volume',
                  'Dividends', 'Stock Splits',
                  'Returns', 'Unnamed: 0', 'index', 'symbol'], axis=1, inplace =True)
    return

  def train_test_split(self):
    X = self.df.drop(['Returns_binary'], axis=1).copy()
    y = self.df['Returns_binary'].copy()
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X,
                                                                            y,
                                                                            test_size=0.33,
                                                                            random_state=42)

    return




  def LR_class(self, naive=True, metric = "f1_weighted"):

    ## Running StandardScaler() for Loogistic Regression
    from sklearn import preprocessing
    scaler = preprocessing.StandardScaler().fit(self.X_train)
    X_scaled = scaler.transform(self.X_train)
    self.X_train = X_scaled

    ##run basic model
    if naive == True:
      LR_final = LogisticRegression()
      scores = cross_val_score(LR_final, self.X_train, self.y_train,
                              cv=5 , scoring=metric)
      self.LR_cv_results = scores
      print("Results of basic Logistic Regression:")
      print('Metric: ', metric, " mean score of cross val: ", scores.mean())

    else:
        ##initiation of optuna for hyperparameter tunning

        ##objective function for maximisation
        def objective(trial):

          params = {
              'tol' : trial.suggest_float('tol' , 1e-6 , 1e-3),
              'C' : trial.suggest_float("C", 1e-2, 1, log=True)}
          model = LogisticRegression(**params)
          model.fit(self.X_train, self.y_train)
          y_pred = model.predict(self.X_train)

          score = cross_val_score(model, self.X_train, self.y_train,
                                cv=5, scoring="f1_weighted")
          return score.mean()   ##f1_score(self.y_train, y_pred)

        sampler = TPESampler(seed=42)
        study = optuna.create_study(direction='maximize', sampler=sampler)
        study.optimize(objective, n_trials=40)


        print('the best params:' , study.best_trial.params)

        ##feed best parameters for cross validation
        params_final = study.best_trial.params
        LR_final = LogisticRegression(**params_final)


        scores = cross_val_score(LR_final, self.X_train, self.y_train,
                                cv=5 , scoring="f1_weighted")

        self.LR_cv_results = scores
        ##Report
        print("Results of hyper-tuned Logistic Regression:")
        print('Metric: ', metric, " mean score of cross val: ", scores.mean())
        return




  def RF_class(self, naive=True, metric = "f1_weighted"):

    ##run basic model
    if naive == True:
      LR_final = RandomForestClassifier()
      scores = cross_val_score(LR_final, self.X_train, self.y_train,
                              cv=5 , scoring=metric)
      self.LR_cv_results = scores
      print("Results of basic Random Forest:")
      print('Metric: ', metric, " mean score of cross val: ", scores.mean())
      return
    else:
      ##initiation of optuna for hyperparameter tunning

      ##objective function for maximisation
      def objective(trial):

        params = {
            'n_estimators' : trial.suggest_int('tol' , 5, 200)}
        model = RandomForestClassifier(**params)
        model.fit(self.X_train, self.y_train)
        y_pred = model.predict(self.X_train)
        scores = cross_val_score(model, self.X_train, self.y_train,
                              cv=5, scoring="f1_weighted")
        return scores.mean()

      sampler = TPESampler(seed=123)
      study = optuna.create_study(direction='maximize', sampler=sampler)
      study.optimize(objective, n_trials=20)


      print('the best params:' , study.best_trial.params)
      params_final = study.best_trial.params

      ##feed best parameters for cross validation
      RF_final = LogisticRegression(**params_final)


      scores = cross_val_score(RF_final, self.X_train, self.y_train,
                              cv=5, scoring="f1_weighted")
      ##report
      self.RF_cv_results = scores
      print("Results of hyper-tuned Random Forest:")
      print('Metric: ', metric, " mean score of cross val: ", scores.mean())
      return




In [ ]:
## Last run of class for basic models
ML = ML_build()

ML.news_df(News_dataset.head(10)) ##init with small size
ML.load_BERT() ## load df with current features


##encoding specific variables
ML.One_hot_encode()

##Import stock data and merge data on Ticker + Date
ML.import_stock_data()
ML.prepare_final()

##train-test split with fixed random state = 42
ML.train_test_split()

<ipython-input-104-6dffe982a776>:43: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  self.df_news.loc[:,'Date'] = pd.to_datetime(self.df_news.loc[:,'Date'])
<ipython-input-104-6dffe982a776>:397: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Stock_data = Stock_data.append(hist)
<ipython-input-104-6dffe982a776>:397: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Stock_data = Stock_data.append(hist)
<ipython-input-104-6dffe982a776>:397: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Stock_data = Stock_data.a

In [ ]:
##Basic model results
ML.LR_class()
ML.RF_class()

Results of basic Logistic Regression:
Metric:  f1_weighted  mean score of cross val:  0.769856749946499
Results of basic Random Forest:
Metric:  f1_weighted  mean score of cross val:  0.7918631458875477


In [ ]:
## Hyperparameter tuning
ML.LR_class(False)


[I 2023-11-29 20:10:16,012] A new study created in memory with name: no-name-5fd211d7-145b-4bee-b11a-13deb63fe7df
[I 2023-11-29 20:10:16,261] Trial 0 finished with value: 0.7704077411807163 and parameters: {'tol': 0.0003751655787285152, 'C': 0.7969454818643931}. Best is trial 0 with value: 0.7704077411807163.
[I 2023-11-29 20:10:16,458] Trial 1 finished with value: 0.7744384194414999 and parameters: {'tol': 0.0007322619478695938, 'C': 0.15751320499779725}. Best is trial 1 with value: 0.7744384194414999.
[I 2023-11-29 20:10:16,593] Trial 2 finished with value: 0.7696964721021048 and parameters: {'tol': 0.0001568626218019941, 'C': 0.020511104188433976}. Best is trial 1 with value: 0.7744384194414999.
[I 2023-11-29 20:10:16,791] Trial 3 finished with value: 0.7709731599736158 and parameters: {'tol': 5.9025528556031264e-05, 'C': 0.5399484409787431}. Best is trial 1 with value: 0.7744384194414999.
[I 2023-11-29 20:10:16,924] Trial 4 finished with value: 0.7728066046394465 and parameters: {'

the best params: {'tol': 0.000501031458251255, 'C': 0.17262825947776445}
Results of hyper-tuned Logistic Regression:
Metric:  f1_weighted  mean score of cross val:  0.7744649750847148


In [ ]:
## Hyperparameter tuning
ML.RF_class(False)

[I 2023-11-29 20:10:21,346] A new study created in memory with name: no-name-39dae0e4-d367-4462-8d88-1801f8802efb
[I 2023-11-29 20:10:25,435] Trial 0 finished with value: 0.783890040712416 and parameters: {'tol': 141}. Best is trial 0 with value: 0.783890040712416.
[I 2023-11-29 20:10:27,228] Trial 1 finished with value: 0.7775397490818842 and parameters: {'tol': 61}. Best is trial 0 with value: 0.783890040712416.
[I 2023-11-29 20:10:28,690] Trial 2 finished with value: 0.7833882261593119 and parameters: {'tol': 49}. Best is trial 0 with value: 0.783890040712416.
[I 2023-11-29 20:10:32,057] Trial 3 finished with value: 0.7873664607275785 and parameters: {'tol': 113}. Best is trial 3 with value: 0.7873664607275785.
[I 2023-11-29 20:10:36,598] Trial 4 finished with value: 0.7844875676610357 and parameters: {'tol': 146}. Best is trial 3 with value: 0.7873664607275785.
[I 2023-11-29 20:10:39,543] Trial 5 finished with value: 0.7849272757621281 and parameters: {'tol': 87}. Best is trial 3 w

the best params: {'tol': 113}
Results of hyper-tuned Random Forest:
Metric:  f1_weighted  mean score of cross val:  0.7584119331311427


In [ ]:
## NEWS dataset
ML.df_news

,Unnamed: 0.1,Unnamed: 0,index,Title,link,Date,Ex.,Company,FinBERT,Fin_BERT_score,...,B-LOC,I-LOC,I-MISC,negative,neutral,positive,LABEL_0,LABEL_1,positive*Label_1,negative*Label_1
0,0,0,0,Apple and Imagination Technologies Bury the Ha...,https://news.google.com/rss/articles/CBMiQ2h0d...,2020-01-02,NASDAQ,AAPL,neutral,0.840931,...,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1,1,1,Apple's stock could be worth $100 more in 2020...,https://news.google.com/rss/articles/CBMiZmh0d...,2020-01-01,NASDAQ,AAPL,neutral,0.873386,...,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,2,2,2,Apple restores Imagination GPU chip agreement ...,https://news.google.com/rss/articles/CBMiO2h0d...,2020-01-01,NASDAQ,AAPL,positive,0.587373,...,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,3,3,3,"Demolition underway at Apple Valley, Summitvie...",https://news.google.com/rss/articles/CBMirwFod...,2020-01-01,NASDAQ,AAPL,neutral,0.919252,...,"['Apple', 'Valley', 'Summit', '##view']","['Apple', 'Valley', 'Summit', '##view']",NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,4,4,4,Want Fresh Fruit in Winter? Try Crab Apple Trees,https://news.google.com/rss/articles/CBMiS2h0d...,2023-06-29,NASDAQ,AAPL,neutral,0.926606,...,NaN,NaN,"['##rab', 'Apple', 'Trees']",0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,121754,22202,"DeepMind launches new AI tool, Dramatron, to w...",https://news.google.com/rss/articles/CBMif2h0d...,2022-12-12,NASDAQ,GOOGL,neutral,0.880453,...,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0
99996,99996,121755,22203,Google's Quest to Digitize Valuable Military T...,https://news.google.com/rss/articles/CBMiQ2h0d...,2022-12-12,NASDAQ,GOOGL,neutral,0.931496,...,NaN,NaN,"['Google', 'Quest', 'to', '##ize', 'Valuable',...",0.0,1.0,0.0,0.0,1.0,0.0,0.0
99997,99997,121756,22204,Ahsan Khan Shows His Newly Engraved Tattoo,https://news.google.com/rss/articles/CBMiP2h0d...,2022-12-12,NASDAQ,GOOGL,neutral,0.643163,...,NaN,NaN,"['Ah', '##san', 'Khan', 'Ta', '##tto']",0.0,1.0,0.0,0.0,1.0,0.0,0.0
99998,99998,121757,22205,"The Mystery, Magic and Meaning of Viking Runes",https://news.google.com/rss/articles/CBMiT2h0d...,2022-12-12,NASDAQ,GOOGL,neutral,0.910372,...,NaN,NaN,"['Mystery', ',', 'Magic', 'and', 'Mean', '##in...",0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
##Final df
ML.df

,SP500_returns,SP500_returns_yesterday,Returns_binary,Returns_yesterday,Unnamed: 0.1,Fin_BERT_score,FakeBERT_score,negative,neutral,positive,LABEL_0,LABEL_1,positive*Label_1,negative*Label_1,Uncert,positive*Label_1*Uncert,negative*Label_1*Uncert
Date,,,,,,,,,,,,,,,,,
2020-01-02,0.004062,0.002631,1,0.000942,690871,38.650980,47.983754,3.0,38.0,7.0,32.0,16.0,4.0,0.0,0.642857,2.571429,0.000000
2020-01-03,0.002631,0.008929,1,0.020457,484683,33.862168,40.928258,4.0,31.0,6.0,26.0,15.0,0.0,2.0,0.642857,0.000000,1.285714
2020-01-06,0.008929,-0.001444,1,-0.004836,393998,33.524743,38.118573,8.0,29.0,2.0,30.0,9.0,0.0,0.0,0.266667,0.000000,0.000000
2020-01-07,-0.001444,0.004465,0,0.020292,463193,27.656916,32.560617,4.0,28.0,1.0,26.0,7.0,1.0,1.0,0.500000,0.500000,0.500000
2020-01-08,0.004465,0.002655,1,0.007779,488291,40.200368,45.975963,3.0,40.0,3.0,35.0,11.0,1.0,0.0,0.357143,0.357143,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-21,-0.003037,0.002476,0,-0.001069,84141,0.902314,0.999677,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.470588,0.000000,0.000000
2023-07-25,0.002694,0.001709,1,-0.006151,90676,0.793166,0.999788,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.508772,0.000000,0.000000
2023-07-26,0.001709,-0.013233,0,-0.017240,84546,0.930752,0.999789,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.278689,0.000000,0.000000


In [ ]:
ML.df.describe().to_excel("Final_df_description.xlsx")

In [ ]:
## Theoretically Full iteration of class in one run


## init class and import news df
ML = ML_build()
ML.news_df(News_dataset)

## Drop Non-english, duplicates and other
ML.data_preparation()


## Run DBSCAN model
ML.first_story_flag() ##WIP. Breaks on large dataset

## Run ML moddels
ML.BERT_sentiment_test()
ML.BERT_quality_test()
ML.BERT_NER_test()


##Save current changes to news dataframe
ML.save_BERT()
ML.load_BERT()

## Date adjustmens
ML.time_publication_adjustment(True) ##WIP. Non-naive (false) breaks on large dataset

##encoding specific variables
ML.One_hot_encode()

##Import stock data and merge data on Ticker + Date
ML.import_stock_data()
ML.prepare_final()

##train-test split with fixed random state = 42
ML.train_test_split()



In [ ]:
ML = ML_build()
ML.news_df(News_dataset)
ML.data_preparation()

ML.BERT_sentiment_test()
ML.BERT_quality_test()
##ML.first_story_flag()
ML.save_BERT()


<ipython-input-41-b335f4cacec8>:30: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  self.df_news.loc[:,'Date'] = pd.to_datetime(self.df_news.loc[:,'Date'])
5206366it [1:21:44, 1061.65it/s]


{'label': 'neutral', 'score': 0.8648032546043396}


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

5206366it [1:21:27, 1065.18it/s]


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


  2%|▏         | 87412/5206366 [13:15<12:56:19, 109.90it/s]


KeyboardInterrupt: ignored

####Data overveiw (Submission 2)

In [ ]:
# install the sweetviz package
!pip install sweetviz
# load the sweetviz
import sweetviz
# use analyze
analyze_df = sweetviz.analyze([ML.df, "df"], target_feat = 'Returns_binary')
# then show
analyze_df.show_html('analyze.html')

####Discontinued functions

In [ ]:
  def BERT_sentiment(self, column = 'Title' ):
    from transformers import pipeline
    nlp = pipeline(task='sentiment-analysis',
               model='ProsusAI/finbert', device=0)

    def classifier(text):
      return nlp(text)
      return f'{nlp(text)}'

    ## Cool code from stackoverflow https://stackoverflow.com/questions/64342621/how-to-apply-a-sentiment-classifier-to-a-dataframe
    self.df_news = (
    self.df_news
    .assign(sentiment = lambda x: x[column].progress_apply(lambda s: classifier(s)))
    .assign(
         label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
         score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))))

In [ ]:

  def BERT_quality(self, column = 'Title' ):
    from transformers import pipeline
    nlp = pipeline("text-classification", model="ikoghoemmanuell/finetuned_fake_news_roberta",
                   device=0)

    def classifier(text):
      return nlp(text)
      return f'{nlp(text)}'

    ## Cool code from stackoverflow https://stackoverflow.com/questions/64342621/how-to-apply-a-sentiment-classifier-to-a-dataframe
    self.df_news = (
    self.df_news
    .assign(sentiment = lambda x: x[column].progress_apply(lambda s: classifier(s)))
    .assign(
         Fake = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
         score_fake = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))))
    return





